# Sentiment Analysis based on Attention Mechanism

# Train models (Inter-Attention-BiLSTM & Self-Inter-Attention-BiLSTM) on Four Datasets

## Dataset
- [**IMDB Large Movie Review Dataset**](http://ai.stanford.edu/~amaas/data/sentiment/)
    - **Binary** sentiment classification
    - Citation: [Andrew L. Maas et al., 2011](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf)
    - **50,000** movie reviews for training and testing
    - Average review length: **231** vocab
    ---
- [**Yelp reviews-full**](http://xzh.me/docs/charconvnet.pdf)
    - **Multiclass** sentiment classification (5 stars)
    - Citation: [Xiang Zhang et al., 2015](https://arxiv.org/abs/1509.01626)
    - **650,000** training samples and **50,000** testing samples (Nums of each star are equal)
    - Average review length: **140** vocab
    ---
- [**Yelp reviews-polarity**](http://xzh.me/docs/charconvnet.pdf)
    - **Binary** sentiment classification
    - Citation: [Xiang Zhang et al., 2015](https://arxiv.org/abs/1509.01626)
    - **560,000** training samples and **38,000** testing samples (Nums of positive or negative samples are equal)
    - Average review length: **140** vocab
    ---
- [**Douban Movie Reviews**](https://drive.google.com/open?id=1DsmQfB1Ff_BUoxOv4kfUMg7Y8M7tHB9F) 
    - My **Custom Chinese** movie reviews scraped from **16000** movies (num of reviews > 100) on [Douban](https://movie.douban.com/)
    - **Binary** sentiment classification
    - **700,000** movie reviews , **600,000** samples for training and **100,000** samples for testing (Num of positive or negative samples is equal)
    - Average review length: **50** character, **27** vocab

## paper reference

> [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/pdf/1409.0473.pdf)
>
> [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/pdf/1508.04025.pdf)
>
> [Attention Is All You Need](https://arxiv.org/abs/1706.03762)

## code reference
> https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation-batched.ipynb
>
> https://github.com/graykode/nlp-tutorial/blob/master/4-3.Bi-LSTM(Attention)/Bi_LSTM(Attention)_Torch.ipynb
>
> https://github.com/bojone/attention/blob/master/attention_keras.py
>
> https://github.com/graykode/nlp-tutorial/blob/master/5-1.Transformer/Transformer(Greedy_decoder)_Torch.ipynb

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import DataLoader, Dataset, SequentialSampler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import time
import random
import os
import copy
import warnings

from utils import pad_and_sort_batch, preprocess_for_batch, pad_or_truncate

warnings.filterwarnings('ignore')
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

%matplotlib inline
%load_ext autoreload
%autoreload 2
torch.__version__

'1.0.1.post2'

In [0]:
# set random seeds to keep the results identical
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    
def worker_init_fn(worker_id):
    setup_seed(torch.initial_seed() + worker_id)
    
GLOBAL_SEED = 2019
setup_seed(GLOBAL_SEED)

In [0]:
base_dir = './'

In [0]:
# setting in google colab
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/'

In [1]:
# pre setting in google colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!pip install -U pandas
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
    100% |████████████████████████████████| 10.1MB 4.3MB/s 
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## Load train data (Choose one of the four datasets to train)

In [0]:
#data_pth = base_dir + 'dataset/IMDB/'
#data_pth = base_dir + 'dataset/yelp_review_polarity_csv/'
#data_pth = base_dir + 'dataset/yelp_review_full_csv/'
data_pth = base_dir + 'dataset/Douban/'

In [0]:
X_train = pd.read_hdf(data_pth+'X_train.h5', key='s')
y_train = pd.read_hdf(data_pth+'y_train.h5', key='s')

X_val = pd.read_hdf(data_pth+'X_val.h5', key='s')
y_val = pd.read_hdf(data_pth+'y_val.h5', key='s')

word2num_series = pd.read_hdf(data_pth+'word2num_series.h5', key='s')

In [7]:
len(X_train)

550000

In [8]:
len(X_val)

50000

## Load pretrained word embedding matrix (Glove)
- Glove https://github.com/stanfordnlp/GloVe
- Chinese Word Vector https://github.com/Embedding/Chinese-Word-Vectors

In [0]:
# English
# pre_embedding = {}
# with open('/content/gdrive/My Drive/Colab Notebooks/word2vector/glove.twitter.27B.200d.txt', encoding='utf8') as f:
#     for line in f.readlines():
#         tmp = line.strip().split()
#         if tmp[0] in word2num_series:
#             pre_embedding[tmp[0]] = np.array(tmp[1:]).astype(np.float)

# Chinese
pre_embedding = {}
with open(base_dir + 'word2vector/sgns.weibo.bigram-char', encoding='utf8') as f:
    for line in f.readlines():
        tmp = line.strip().split()
        if tmp[0] in word2num_series:
            pre_embedding[tmp[0]] = np.array(tmp[1:]).astype(np.float)

In [10]:
vocab_size = len(word2num_series)+3
dim = pre_embedding['movie'].shape[0]
print('dimention:', dim)
mean = np.mean([value for _, value in pre_embedding.items()])
std = np.std([value for _, value in pre_embedding.items()])
print('mean:', np.mean([value for _, value in pre_embedding.items()]))
print('std:', np.std([value for _, value in pre_embedding.items()]))
print('max:', np.max([value for _, value in pre_embedding.items()]))
print('min:', np.min([value for _, value in pre_embedding.items()]))

dimention: 300
mean: 0.009304564211605073
std: 0.398704565783848
max: 2.5235
min: -2.452009


In [0]:
embedding_matrix = np.random.randn(vocab_size, dim)*std

In [0]:
miss_word = 0
for word, idx in word2num_series.items():
    try:
        embedding_matrix[idx] = pre_embedding[word]
    except:
        miss_word += 1
        #print(word)

In [13]:
miss_word

7471

In [0]:
np.testing.assert_array_almost_equal(embedding_matrix[word2num_series['movie']], pre_embedding['movie'])

## Build pytorch dataset and dataloader

In [0]:
BATCH_SIZE = 128

In [0]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        if self.features[idx].size(0) > 0:
            features = self.features[idx]
        else:
            features = torch.LongTensor([1])
        return features, self.labels[idx], max(len(self.features[idx]), 1)

### Sort the train data by sentence length which aims to reduce 0 padding in every batch, then shuffle batchs

In [0]:
# convert list item to tensor, avoid memory leak in dataset
# There was a bug fixed here, X_train_sorted and y_train_sorted must be tensor type, not python native list(can cause memory leak)
# https://github.com/pytorch/pytorch/issues/13246

############ just for multiclass #############
# y_train = y_train-1
# y_val = y_val-1
##############################################

X_train_sorted_tensors, y_train_sorted_tensors = preprocess_for_batch(X_train, y_train, BATCH_SIZE) # ignore last batch if smaller than batch_size
X_valid_tensors = [torch.tensor(x, dtype=torch.int64) for x in X_val]
y_valid_tensors = [torch.tensor(y, dtype=torch.int64) for y in y_val]

In [0]:
# truncate long sentence's length to maxlen
########### just for Transformer model ##################
MAXLEN = 500
X_train_sorted_tensors = [pad_or_truncate(x, maxlen=MAXLEN, pad=False) for x in X_train_sorted_tensors] # just truncate no padding to save GPU memory
X_valid_tensors = [pad_or_truncate(x, maxlen=MAXLEN, pad=False) for x in X_valid_tensors]

In [0]:
train_dataset = CustomDataset(X_train_sorted_tensors, y_train_sorted_tensors)
val_dataset = CustomDataset(X_valid_tensors, y_valid_tensors)

In [20]:
print("train-set size:", len(train_dataset))
print("valid-set size:", len(val_dataset))

train-set size: 549888
valid-set size: 50000


In [0]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=SequentialSampler(train_dataset), shuffle=False, collate_fn=pad_and_sort_batch, worker_init_fn=worker_init_fn)
valid_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=pad_and_sort_batch, worker_init_fn=worker_init_fn)



## Train the model

In [0]:
from SIAttentionBiLSTM import SIAttentionBiLSTM
from AvgBiLSTM import AvgBiLSTM
from Transformer import Transformer

### set model hyperparameters

In [0]:
#AvgBiLSTM
# parameters = {
#     'hidden_size': 256,
#     'embedding': embedding_matrix,
#     'rnn_dropout': 0.2,
#     'embedding_dropout': 0.6,
# }

#InterAttentionBiLSTM
# parameters = {
#     'hidden_size': 256,
#     'embedding': embedding_matrix,
#     'att_method': 'concat',
#     'rnn_dropout': 0.3,
#     'embedding_dropout': 0.6,
#     'context_dropout': 0.2
# }

# SIAttention-BiLSTM
# parameters = {
#     'hidden_size': 128,
#     'embedding': embedding_matrix,
#     'att_method': 'concat',
#     'n_layers': 1,
#     'd_model': 256, 
#     'n_heads': 4, 
#     'd_k': 64, 
#     'd_v': 64, 
#     'rnn_dropout': 0.1,
#     'transformer_dropout':0.1,
#     'embedding_dropout': 0.5,
#     'context_dropout': 0.1,
#     'multihead_dropout': 0.1,
#     'inter_att_dropout':0.1,
#     'self_att_dropout': 0.1
# }

# Transformer
parameters = {
    'n_layers': 3,
    'd_model': 300, 
    'd_ff':512, 
    'n_heads': 5, 
    'd_k': 60, 
    'd_v': 60, 
    'final_att_method': 'concat',
    'embedding': embedding_matrix,
    'embedding_dropout': 0.5,
    'multihead_dropout': 0.1,
    'att_dropout': 0.1,
    'feedforward_dropout': 0.1,
    'final_dropout': 0.1
}


In [24]:
# Polarity
#model = AvgBiLSTM(output_size=1, hidden_size=parameters['hidden_size'], embedding=parameters['embedding'], rnn_dropout=parameters['rnn_dropout'], embedding_dropout=parameters['embedding_dropout'])
#model = SIAttentionBiLSTM(n_layers=parameters['n_layers'], d_model=parameters['d_model'], d_k=parameters['d_k'], d_v=parameters['d_v'], n_heads=parameters['n_heads'], hidden_size=parameters['hidden_size'], embedding=parameters['embedding'], method=parameters['att_method'], rnn_dropout=parameters['rnn_dropout'], embedding_dropout=parameters['embedding_dropout'], multihead_dropout=parameters['multihead_dropout'], context_dropout=parameters['context_dropout'], transformer_dropout=parameters['transformer_dropout'], inter_att_dropout=parameters['inter_att_dropout'], self_att_dropout=parameters['self_att_dropout'])

architecture = (parameters['n_layers'], parameters['d_model'], parameters['d_ff'], parameters['n_heads'], parameters['d_k'], parameters['d_v'])
model = Transformer(output_size=1, architecture=architecture, embedding=parameters['embedding'], method=parameters['final_att_method'], maxpos=MAXLEN, embedding_dropout=parameters['embedding_dropout'], multihead_dropout=parameters['multihead_dropout'], att_dropout=parameters['att_dropout'], feedforward_dropout=parameters['feedforward_dropout'], final_dropout=parameters['final_dropout'])


# Full
#model = AvgBiLSTM(output_size=5, hidden_size=parameters['hidden_size'], embedding=parameters['embedding'], rnn_dropout=parameters['rnn_dropout'], embedding_dropout=parameters['embedding_dropout'])
#model = AttnBiLSTM(output_size=5, hidden_size=parameters['hidden_size'], embedding=parameters['embedding'], method=parameters['att_method'], rnn_dropout=parameters['rnn_dropout'], embedding_dropout=parameters['embedding_dropout'], context_dropout=parameters['context_dropout'])
#model = SIAttentionBiLSTM(output_size=5, n_layers=parameters['n_layers'], d_model=parameters['d_model'], d_k=parameters['d_k'], d_v=parameters['d_v'], n_heads=parameters['n_heads'], hidden_size=parameters['hidden_size'], embedding=parameters['embedding'], method=parameters['att_method'], rnn_dropout=parameters['rnn_dropout'], embedding_dropout=parameters['embedding_dropout'], multihead_dropout=parameters['multihead_dropout'], context_dropout=parameters['context_dropout'], transformer_dropout=parameters['transformer_dropout'], inter_att_dropout=parameters['inter_att_dropout'], self_att_dropout=parameters['self_att_dropout'])

model.to(DEVICE)

Transformer(
  (pos_add_word_embedding): PE_add_Embedding(
    (word_embedding): Embedding(40000, 300)
    (pos_embedding): Embedding(501, 300)
    (dropout): Dropout(p=0.5)
  )
  (encode_layers): ModuleList(
    (0): EncoderLayer(
      (multihead_attention): MultiHeadAttention(
        (linear_Q): Linear(in_features=300, out_features=300, bias=True)
        (linear_K): Linear(in_features=300, out_features=300, bias=True)
        (linear_V): Linear(in_features=300, out_features=300, bias=True)
        (dropout): Dropout(p=0.1)
        (attn): ScaleDotProductAttention(
          (dropout): Dropout(p=0.1)
        )
        (out_linear): Linear(in_features=300, out_features=300, bias=True)
        (layer_norm): LayerNorm(torch.Size([300]), eps=1e-05, elementwise_affine=True)
      )
      (postion_feedforward): PoswiseFeedForward(
        (conv1): Conv1d(300, 512, kernel_size=(1,), stride=(1,))
        (conv2): Conv1d(512, 300, kernel_size=(1,), stride=(1,))
        (dropout): Dropout(p=

In [0]:
# Polarity
criterion = nn.BCEWithLogitsLoss()

# Full
#criterion = nn.CrossEntropyLoss()


# param_optimizer = list(model.named_parameters())
# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight', 'lstm_encoder.embedding.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.00001},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#     ]


optimizer = torch.optim.Adam(model.parameters())
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.9, last_epoch=-1)

In [0]:
def validate(model, criterion, history):
    model.eval()
    global best_acc, best_model, validate_history
    costs = []
    accs = []
    with torch.no_grad():
        for idx, batch in enumerate(valid_dataloader):
            input_batch, labels, lengths = batch
            labels = labels.float().unsqueeze(1) # polarity
            #output = model(input_batch, lengths) # RNN
            output = model(input_batch)  # Transformer
            loss = criterion(output, labels)
            costs.append(loss.item())
            # polarity
            accs.append((output>0).eq(labels>0).float().mean().item())
            
            # full
            #_, preds = torch.max(output, 1)
            #accs.append((preds == labels).float().mean().item())
            torch.cuda.empty_cache()
    mean_accs = np.mean(accs)
    mean_costs = np.mean(costs)
    if mean_accs > history['best_acc']:  
        history['best_acc'] = mean_accs
        history['best_model'] = copy.deepcopy(model.state_dict())
        
    history['validate_accuracy'].append(mean_accs)
    history['validate_loss'].append(mean_costs)
    return mean_costs, mean_accs


def train(model, criterion, optimizer, epoch, history, validate_points):
    model.train()
    costs = []
    accs = []
    with tqdm(total=len(train_dataset), desc='Epoch {}'.format(epoch)) as pbar:
        for idx, batch in enumerate(train_dataloader):
            input_batch, labels, lengths = batch
            labels = labels.float().unsqueeze(1) # polarity
            #output = model(input_batch, lengths) # RNN
            output = model(input_batch)  # Transformer
            loss = criterion(output, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                costs.append(loss.item())
                # polarity
                accs.append((output>0).eq(labels>0).float().mean().item())

                # full
                #_, preds = torch.max(output, 1)
                #accs.append((preds == labels).float().mean().item())
                pbar.update(input_batch.size(0))
                pbar.set_postfix_str('train-loss: {:.4f}, train-acc: {:.4f}'.format(np.mean(costs), np.mean(accs)))
            if idx in validate_points:
                val_loss, val_acc = validate(model, criterion, history)
                pbar.set_postfix_str('train-loss: {:.4f}, train-acc: {:.4f}, val-loss: {:.4f}, val-acc: {:.4f}'.format(np.mean(costs), np.mean(accs), val_loss, val_acc))
                model.train()
            torch.cuda.empty_cache()
    
    history['train_loss'].append(costs)
    history['train_accuracy'].append(accs)

In [0]:
history = { 'best_acc': 0,
            'best_model': None,
            'optimizer': optimizer.state_dict(),
            'train_accuracy': [],
            'train_loss': [],
            'validate_accuracy': [],
            'validate_loss': [],
            'batch_size': BATCH_SIZE,
            'num_of_batch': len(train_dataloader),
            'train_size': len(train_dataset),
            'validate_size': len(val_dataset),
            'validate_points': None,
            'epochs': 0,
            'embedding_size': embedding_matrix.shape,
            'parameters': parameters
          }

In [0]:
epochs = 10
validate_points = list(np.linspace(0, len(train_dataloader)-1, 4).astype(int))[1:]
history['epochs'] = epochs
history['validate_points'] = validate_points

## Attention Model on Douban dataset

### Transformer

In [39]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Transformer-on-Douban', timestamp))

Epoch 10: 100%|██████████| 549888/549888 [06:54<00:00, 1229.82it/s, train-loss: 0.3894, train-acc: 0.8246, val-loss: 0.4319, val-acc: 0.8071]


CPU times: user 36min 15s, sys: 31min 46s, total: 1h 8min 1s
Wall time: 1h 9min 12s


### AvgBiLSTM

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Avg-BiLSTM-on-Douban', timestamp))

Epoch 10: 100%|██████████| 549888/549888 [03:20<00:00, 2745.24it/s, train-loss: 0.2477, train-acc: 0.8954, val-loss: 0.4635, val-acc: 0.8289]


CPU times: user 17min 29s, sys: 14min 34s, total: 32min 3s
Wall time: 33min 37s


### Self-Attention + Inter-Attention + BiLSTM

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Self-Inter-Attention-BiLSTM-on-Douban', timestamp))

Epoch 10: 100%|██████████| 549888/549888 [04:08<00:00, 2209.32it/s, train-loss: 0.2338, train-acc: 0.9014, val-loss: 0.4686, val-acc: 0.8335]


CPU times: user 22min 29s, sys: 18min 43s, total: 41min 13s
Wall time: 41min 48s


### BiLSTM + Inter-Attention(query=none , key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-no-query-BiLSTM-on-Douban', timestamp))

Epoch 15: 100%|██████████| 549888/549888 [02:35<00:00, 3546.69it/s, train-loss: 0.2215, train-acc: 0.9072, val-loss: 0.4971, val-acc: 0.8327]


CPU times: user 18min 28s, sys: 18min 36s, total: 37min 4s
Wall time: 38min 59s


### BiLSTM + Inter-Attention(query=independent parameter , key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-independent-query-BiLSTM-on-Douban', timestamp))

Epoch 15: 100%|██████████| 549888/549888 [02:53<00:00, 3162.81it/s, train-loss: 0.2215, train-acc: 0.9075, val-loss: 0.4886, val-acc: 0.8317]


CPU times: user 23min 32s, sys: 19min 55s, total: 43min 27s
Wall time: 44min 26s


### BiLSTM + Inter-Attention(query=h_n of BiLSTM, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-h-query-BiLSTM-on-Douban', timestamp))

Epoch 15: 100%|██████████| 549888/549888 [02:51<00:00, 3586.93it/s, train-loss: 0.2204, train-acc: 0.9080, val-loss: 0.4854, val-acc: 0.8345]


CPU times: user 22min 6s, sys: 20min 21s, total: 42min 28s
Wall time: 43min 54s


### BiLSTM + Inter-Attention(query=c_n of BiLSTM, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-c-query-BiLSTM-on-Douban', timestamp))

Epoch 15: 100%|██████████| 549888/549888 [03:08<00:00, 2903.03it/s, train-loss: 0.2216, train-acc: 0.9070, val-loss: 0.4768, val-acc: 0.8333]


CPU times: user 23min 12s, sys: 21min 38s, total: 44min 50s
Wall time: 47min 7s


## Attention Model on Yelp Full dataset

### Transformer

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Transformer-on-Yelp-Full', timestamp))

Epoch 6:  67%|██████▋   | 400000/599936 [19:45<06:16, 530.85it/s, train-loss: 0.8933, train-acc: 0.6110]

### Avg-BiLSTM

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Avg-BiLSTM-on-Yelp-Full', timestamp))

Epoch 10: 100%|██████████| 599808/599808 [09:53<00:00, 1011.05it/s, train-loss: 0.6657, train-acc: 0.7130, val-loss: 1.1007, val-acc: 0.6584]


CPU times: user 40min 23s, sys: 56min 54s, total: 1h 37min 18s
Wall time: 1h 39min 37s


### Self-Attention + Inter-Attention + BiLSTM

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Self-Inter-Attention-BiLSTM-on-Yelp-Full', timestamp))

Epoch 5: 100%|██████████| 599808/599808 [15:18<00:00, 653.09it/s, train-loss: 0.6501, train-acc: 0.7219, val-loss: 0.8485, val-acc: 0.6652] 


CPU times: user 28min 15s, sys: 47min 34s, total: 1h 15min 49s
Wall time: 1h 16min 39s


### BiLSTM + Inter-Attention(query=none, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-no-query-BiLSTM-on-Yelp-Full', timestamp))

Epoch 15: 100%|██████████| 599808/599808 [10:34<00:00, 945.60it/s, train-loss: 0.6330, train-acc: 0.7291, val-loss: 0.8027, val-acc: 0.6682] 


CPU times: user 1h 1min 53s, sys: 1h 33min 47s, total: 2h 35min 41s
Wall time: 2h 38min 45s


### BiLSTM + Inter-Attention(query=independent parameter , key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time())) 
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-independent-query-BiLSTM-on-Yelp-Full', timestamp))

Epoch 15: 100%|██████████| 599808/599808 [14:41<00:00, 680.43it/s, train-loss: 0.6312, train-acc: 0.7299, val-loss: 0.8013, val-acc: 0.6716] 


CPU times: user 1h 23min 54s, sys: 2h 13min 27s, total: 3h 37min 22s
Wall time: 3h 40min 5s


### BiLSTM + Inter-Attention(query=h_n of BiLSTM, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-h-query-BiLSTM-on-Yelp-Full', timestamp))

Epoch 15: 100%|██████████| 599808/599808 [14:51<00:00, 673.08it/s, train-loss: 0.6263, train-acc: 0.7311, val-loss: 0.8020, val-acc: 0.6686] 


CPU times: user 1h 23min 46s, sys: 2h 14min 24s, total: 3h 38min 11s
Wall time: 3h 43min 55s


### BiLSTM + InterAttention(query=c_n of BiLSTM, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-c-query-BiLSTM-on-Yelp-Full', timestamp))

Epoch 15: 100%|██████████| 599808/599808 [14:57<00:00, 667.96it/s, train-loss: 0.6279, train-acc: 0.7306, val-loss: 0.7951, val-acc: 0.6699] 


CPU times: user 1h 25min 6s, sys: 2h 12min 20s, total: 3h 37min 26s
Wall time: 3h 41min 12s


## Attention Model on Yelp Polarity dataset

### Transformer

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Transformer-on-Yelp-Polarity', timestamp))

Epoch 9:  66%|██████▌   | 341376/519936 [17:06<05:55, 502.41it/s, train-loss: 0.1782, train-acc: 0.9294]

Buffered data was truncated after reaching the output size limit.

### Avg-BiLSTM

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Avg-BiLSTM-on-Yelp-Polarity', timestamp))

Epoch 10: 100%|██████████| 519936/519936 [08:32<00:00, 1774.82it/s, train-loss: 0.0571, train-acc: 0.9793, val-loss: 0.2858, val-acc: 0.9638]


CPU times: user 33min 44s, sys: 49min 19s, total: 1h 23min 4s
Wall time: 1h 25min 12s


In [0]:
history['best_acc']

0.9651671974522293

### Self-Attention + Inter-Attention + BiLSTM

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Self-Inter-Attention-BiLSTM-on-Yelp-Polarity', timestamp))

Epoch 10: 100%|██████████| 519936/519936 [16:06<00:00, 1396.36it/s, train-loss: 0.0484, train-acc: 0.9827, val-loss: 0.1521, val-acc: 0.9626]


CPU times: user 57min 8s, sys: 1h 42min 50s, total: 2h 39min 59s
Wall time: 2h 41min 47s


### BiLSTM + Inter-Attention(query=none, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epo6ch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-no-query-BiLSTM-on-Yelp-Polarity', timestamp))

Epoch 15: 100%|██████████| 519936/519936 [09:31<00:00, 1721.12it/s, train-loss: 0.0428, train-acc: 0.9844, val-loss: 0.1361, val-acc: 0.9616]


CPU times: user 56min 14s, sys: 1h 23min 4s, total: 2h 19min 19s
Wall time: 2h 22min 57s


### BiLSTM + Inter-Attention(query=independent parameter , key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-independent-query-BiLSTM-on-Yelp-Polarity', timestamp))

Epoch 15: 100%|██████████| 519936/519936 [12:51<00:00, 1370.79it/s, train-loss: 0.0434, train-acc: 0.9842, val-loss: 0.1246, val-acc: 0.9635]


CPU times: user 1h 14min 58s, sys: 1h 55min 18s, total: 3h 10min 17s
Wall time: 3h 12min 56s


### BiLSTM + Inter-Attention(query=h_n of BiLSTM, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-h-query-BiLSTM-on-Yelp-Polarity', timestamp))

Epoch 15: 100%|██████████| 519936/519936 [12:30<00:00, 1416.47it/s, train-loss: 0.0407, train-acc: 0.9854, val-loss: 0.1263, val-acc: 0.9642]


CPU times: user 1h 10min 56s, sys: 1h 52min 58s, total: 3h 3min 55s
Wall time: 3h 7min 41s


In [0]:
history['best_acc']

0.9661375398089171

### BiLSTM + Inter-Attention(query=c_n of BiLSTM, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-c-query-BiLSTM-on-Yelp-Polarity', timestamp))

Epoch 15: 100%|██████████| 519936/519936 [12:31<00:00, 1409.37it/s, train-loss: 0.0410, train-acc: 0.9852, val-loss: 0.1261, val-acc: 0.9648]


CPU times: user 1h 11min 55s, sys: 1h 52min 7s, total: 3h 4min 2s
Wall time: 3h 7min 36s


In [0]:
history['best_acc']

0.9661624203821656

## Attention Model on IMDB dataset

### Transformer

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Transformer-on-IMDB', timestamp))

Epoch 10: 100%|██████████| 39936/39936 [04:30<00:00, 134.15it/s, train-loss: 0.1563, train-acc: 0.9398, val-loss: 0.2636, val-acc: 0.9066]


CPU times: user 17min 1s, sys: 27min 42s, total: 44min 44s
Wall time: 45min 8s


### Avg-BiLSTM

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Avg-BiLSTM-on-IMDB', timestamp))

Epoch 15: 100%|██████████| 39936/39936 [01:10<00:00, 551.00it/s, train-loss: 0.0602, train-acc: 0.9788, val-loss: 0.2607, val-acc: 0.9141]


CPU times: user 8min, sys: 9min 13s, total: 17min 14s
Wall time: 17min 42s


In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Avg-BiLSTM-on-IMDB', timestamp))

Epoch 15: 100%|██████████| 39936/39936 [01:10<00:00, 545.63it/s, train-loss: 0.0162, train-acc: 0.9939, val-loss: 0.2915, val-acc: 0.9053]


CPU times: user 8min 3s, sys: 9min 14s, total: 17min 18s
Wall time: 17min 46s


### Self-Attention + Inter-Attention + BiLSTM

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Self-Inter-Attention-BiLSTM-on-IMDB', timestamp))

Epoch 10: 100%|██████████| 39936/39936 [02:17<00:00, 309.23it/s, train-loss: 0.0758, train-acc: 0.9738, val-loss: 0.2731, val-acc: 0.9213]


CPU times: user 8min 39s, sys: 14min, total: 22min 39s
Wall time: 22min 55s


In [0]:
history['best_acc']

0.924609375

### BiLSTM + InterAttention(query=none, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-no-query-BiLSTM-on-IMDB', timestamp))

Epoch 15: 100%|██████████| 39936/39936 [01:22<00:00, 477.81it/s, train-loss: 0.0552, train-acc: 0.9812, val-loss: 0.3242, val-acc: 0.9154]


CPU times: user 9min 21s, sys: 11min, total: 20min 22s
Wall time: 20min 49s


In [0]:
history['best_acc']

0.9240234375

### BiLSTM + InterAttention(query=independent parameter , key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-independent-query-BiLSTM-on-IMDB', timestamp))

Epoch 15: 100%|██████████| 39936/39936 [01:50<00:00, 388.34it/s, train-loss: 0.0587, train-acc: 0.9795, val-loss: 0.3104, val-acc: 0.9187]


CPU times: user 11min 45s, sys: 15min 31s, total: 27min 17s
Wall time: 27min 39s


In [0]:
history['best_acc']

0.92265625

### BiLSTM + InterAttention(query=h_n of BiLSTM, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-h-query-BiLSTM-on-IMDB', timestamp))

Epoch 15: 100%|██████████| 39936/39936 [01:51<00:00, 390.98it/s, train-loss: 0.0487, train-acc: 0.9824, val-loss: 0.3315, val-acc: 0.9158]


CPU times: user 11min 45s, sys: 15min 45s, total: 27min 30s
Wall time: 28min 3s


In [0]:
history['best_acc']

0.9263671875

### BiLSTM + Inter-Attention(query=c_n of BiLSTM, key=value=BiLSTM output)

In [0]:
%%time
timestamp = time.strftime('%Y-%m-%d-%H-%M',time.localtime(time.time()))
for epoch in range(1, epochs+1):
    train(model, criterion, optimizer, epoch, history, validate_points)
    torch.save(history, '{}model save/{}-{}.pth'.format(base_dir, 'Attention-c-query-BiLSTM-on-IMDB', timestamp))

Epoch 15: 100%|██████████| 39936/39936 [01:51<00:00, 384.29it/s, train-loss: 0.0484, train-acc: 0.9827, val-loss: 0.3566, val-acc: 0.9189]


CPU times: user 11min 44s, sys: 15min 39s, total: 27min 24s
Wall time: 27min 56s


In [0]:
history['best_acc']

0.9251953125